In [2]:
# imports
import panel as pn
import dash
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import plotnine as p9
import os
import warnings
from scipy import linalg, optimize
warnings.filterwarnings('ignore')
%matplotlib inline

In [30]:
econ_path = Path('/Users', 'tylerhubbard', 'umnfintech', 'project1_group3', 'Resources', 'FAOSTAT_food_insecurity.csv')
econ_df = pd.read_csv(econ_path, index_col=['Item', 'Area', 'Year'])
econ_df = econ_df.drop(columns=['Domain Code', 'Domain', 'Area Code (FAO)', 'Element Code',
                                'Item Code', 'Year Code', 'Flag', 'Flag Description', 'Note', 'Element'],
                      index=['Average value of food production (constant 2004-2006 I$/cap) (3-year average)',
                      'Cereal import dependency ratio (percent) (3-year average)',
                            'Prevalence of undernourishment (percent) (3-year average)',
                            'Percent of arable land equipped for irrigation (percent) (3-year average)',
                            'Percentage of population using at least basic drinking water services (percent)',
                            'Percentage of population using safely managed drinking water services (Percent)'
                            ])
econ_df.head(60)

Unit  \
Item                                               Area          Year             
Gross domestic product per capita, PPP, dissemi... Cameroon      2010        I$   
                                                                 2011        I$   
                                                                 2012        I$   
                                                                 2013        I$   
                                                                 2014        I$   
                                                                 2015        I$   
                                                                 2016        I$   
                                                                 2017        I$   
                                                                 2018        I$   
                                                                 2019        I$   
                                                                 2020        I$   
Value of food imports in total merchandise expo... Cameroon      2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
Gross domestic product per capita, PPP, dissemi... Côte d'Ivoire 2010        I$   
                                                                 2011        I$   
                                                                 2012        I$   
                                                                 2013        I$   
                                                                 2014        I$   
                                                                 2015        I$   
                                                                 2016        I$   
                                                                 2017        I$   
                                                                 2018        I$   
                                                                 2019        I$   
                                                                 2020        I$   
Value of food imports in total merchandise expo... Côte d'Ivoire 2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
Gross domestic product per capita, PPP, dissemi... Ecuador       2010        I$   
                                                                 2011        I$   
                                                                 2012        I$   
                                                                 2013        I$   
                                                                 2014        I$   
                                                                 2015        I$   
                                                                 2016        I$   
        

In [32]:
gdp_df = econ_df.drop(index='Value of food imports in total merchandise exports (percent) (3-year average)')
food_import_value_df = econ_df.drop(index='Gross domestic product per capita, PPP, dissemination (constant 2011 international $)')

In [33]:
gdp_df

Unit  \
Item                                               Area          Year        
Gross domestic product per capita, PPP, dissemi... Cameroon      2010   I$   
                                                                 2011   I$   
                                                                 2012   I$   
                                                                 2013   I$   
                                                                 2014   I$   
                                                                 2015   I$   
                                                                 2016   I$   
                                                                 2017   I$   
                                                                 2018   I$   
                                                                 2019   I$   
                                                                 2020   I$   
                                                   Côte d'Ivoire 2010   I$   
                                                                 2011   I$   
                                                                 2012   I$   
                                                                 2013   I$   
                                                                 2014   I$   
                                                                 2015   I$   
                                                                 2016   I$   
                                                                 2017   I$   
                                                                 2018   I$   
                                                                 2019   I$   
                                                                 2020   I$   
                                                   Ecuador       2010   I$   
                                                                 2011   I$   
                                                                 2012   I$   
                                                                 2013   I$   
                                                                 2014   I$   
                                                                 2015   I$   
                                                                 2016   I$   
                                                                 2017   I$   
                                                                 2018   I$   
                                                                 2019   I$   
                                                                 2020   I$   
                                                   Ghana         2010   I$   
                                                                 2011   I$   
                                                                 2012   I$   
                                                                 2013   I$   
                                                                 2014   I$   
                                                                 2015   I$   
                                                                 2016   I$   
                                                                 2017   I$   
                                                                 2018   I$   
                                                                 2019   I$   
                                                                 2020   I$   
                                                   Nigeria       2010   I$   
                                                                 2011   I$   
                                                                 2012   I$   
                                                                 2013   I$   
                                                                 2014   I$   
                                                                 2015   I$   
              

In [34]:
food_import_value_df

Unit  \
Item                                               Area          Year             
Value of food imports in total merchandise expo... Cameroon      2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
                                                   Côte d'Ivoire 2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
                                                   Ecuador       2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
                                                   Ghana         2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   
                                                   Nigeria       2009-2011    %   
                                                                 2010-2012    %   
                                                                 2011-2013    %   
                                                                 2012-2014    %   
                                                                 2013-2015    %   
                                                                 2014-2016    %   
                                                                 2015-2017    %   
                                                                 2016-2018    %   
                                                                 2017-2019    %   

                                                                            Value  
Item                                               Area          Year              
Value